In [161]:
# !pip install -U sentence-transformers

In [1]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
import torch
import torch.nn as nn
import nltk
import re
nltk.download('words')
words = set(nltk.corpus.words.words())

def stemSentence_porter(sentence):
    porter = PorterStemmer()
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L 


def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 


def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

def clean_text(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z0-9,.’]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    
    return text

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\czhao\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
movies = pd.read_csv('../tmdb_5000_data/tmdb_5000_movies.csv')
credits = pd.read_csv('../tmdb_5000_data/tmdb_5000_credits.csv') 
movies = movies.merge(credits,on='title')

reviews = pd.read_csv('../crawled_data/2022-11-19_movie_info_with_reviews.csv')
reviews = reviews[['id','reviews']]
movies = movies.merge(reviews,on='id', how='left')

movies.dropna(inplace=True)
movies['release_year'] = movies.release_date.apply(lambda x: x.split("-")[0]).astype(int)
movies = movies[movies['release_year']>1970]

movies = movies[['movie_id','title','release_year','overview','genres','keywords','cast','crew','reviews']]
movies.dropna(inplace=True)

movies['genres'] = movies['genres'].apply(convert)

movies['keywords'] = movies['keywords'].apply(convert)

movies['cast'] = movies['cast'].apply(convert)

movies['cast'] = movies['cast'].apply(lambda x:x[0:3])

movies['crew'] = movies['crew'].apply(fetch_director)


movies['crew'] = movies['crew'].apply(lambda x: [" ".join(\
        w for w in nltk.wordpunct_tokenize(i) \
         if w.lower() in words or not w.isalpha()) for i in x])

# movies['cast'] = movies['cast'].apply(collapse)
# movies['crew'] = movies['crew'].apply(collapse)
# movies['genres'] = movies['genres'].apply(collapse)
# movies['keywords'] = movies['keywords'].apply(collapse)


# new['tags'] = new['tags'].apply(stemSentence_porter)


In [3]:
movies['reviews'] = movies['reviews'].apply(lambda x: clean_text("".join(x)))

In [4]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies['release_year'] = movies['release_year'].astype(str).apply(lambda x:x.split())
movies['tags'] = movies['overview'] + movies['genres'] + movies['release_year'] + movies['keywords'] + movies['cast'] + movies['crew'] 
new = movies.drop(columns=['overview','genres','keywords','cast','crew','release_year'])
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new = new[(new['tags'].notnull())].reset_index(drop=True)

In [5]:
new

,movie_id,title,reviews,tags
0,19995,Avatar,Cameron’s epic can still thrill the audience w...,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,It still might not be quite the conclusion we ...,Following the death of District Attorney Harve...
4,49529,John Carter,John Carter is a good summer movie in March bu...,"John Carter is a war-weary, former military ca..."
...,...,...,...,...
1478,2292,Clerks,Clerks doesn’t feel as if it’s trying too hard...,Convenience and video store clerks Dante and R...
1479,255266,Dry Spell,,Sasha tries to get her soon-to-be ex husband K...
1480,157185,Tin Can Man,"Unique and eerie enough to be worth seeing, ev...",Recently dumped by his girlfirend for another ...
1481,14337,Primer,Time travel may provide the paradoxical mechan...,Friends/fledgling entrepreneurs invent a devic...


In [6]:
new = new[new['reviews']!='']

In [7]:
new

,movie_id,title,reviews,tags
0,19995,Avatar,Cameron’s epic can still thrill the audience w...,"In the 22nd century, a paraplegic Marine is di..."
3,49026,The Dark Knight Rises,It still might not be quite the conclusion we ...,Following the death of District Attorney Harve...
4,49529,John Carter,John Carter is a good summer movie in March bu...,"John Carter is a war-weary, former military ca..."
6,38757,Tangled,"At its finest moments, this production emanate...",When the kingdom's most wanted-and most charmi...
8,767,Harry Potter and the Half-Blood Prince,...a for fans only entry within an often distr...,"As Harry begins his sixth year at Hogwarts, he..."
...,...,...,...,...
1477,42151,Down Terrace,"A film that is compelling in its own right, wh...",After serving jail time for a mysterious crime...
1478,2292,Clerks,Clerks doesn’t feel as if it’s trying too hard...,Convenience and video store clerks Dante and R...
1480,157185,Tin Can Man,"Unique and eerie enough to be worth seeing, ev...",Recently dumped by his girlfirend for another ...
1481,14337,Primer,Time travel may provide the paradoxical mechan...,Friends/fledgling entrepreneurs invent a devic...


In [8]:
# model = SentenceTransformer('paraphrase-distilroberta-base-v1')
# model = SentenceTransformer('all-MiniLM-L6-v2')
# model = SentenceTransformer('all-mpnet-base-v2')
# model = SentenceTransformer('stsb-bert-large')

In [9]:
# def train(model, df, batch_size=128, learning_rate=0.1, epochs=30):

#     model.to(device)
#     model.train()
#     criterion = nn.MSELoss()
#     optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

#     plots = df['tags'].values
    
#     reviews = df['reviews'].values

#     for epoch in range(epochs):
#         for i in range(0, len(plots), batch_size):

#             batch_plots = plots[i:i+batch_size]
#             batch_reviews = reviews[i:i+batch_size]
            
#             embedding_plots = torch.tensor(model.encode(batch_plots), requires_grad=True).unsqueeze(1).to(device)
#             embedding_reviews = torch.tensor(model.encode(batch_reviews), requires_grad=True).unsqueeze(1).to(device)
            
#             loss = criterion(embedding_plots, embedding_reviews)
#             print(loss)
#             loss.backward()
#             optimizer.step()
#             optimizer.zero_grad()

In [10]:
# train(model, new)

In [11]:
# descriptions = new['tags'].tolist()
# des_embeddings = []
# for i,des in enumerate(descriptions):
#     des_embeddings.append(model.encode(des))

In [12]:
# def recommend(query, model):
#     #Compute cosine-similarities with all embeddings 
#     model.eval()
#     query_embedd = model.encode(query)
#     cosine_scores = util.pytorch_cos_sim(query_embedd, des_embeddings)
#     top5_matches = torch.argsort(cosine_scores, dim=-1, descending=True).tolist()[0][0:5]
#     return top5_matches

In [13]:
# query_show_des = 'moon Pandora space colony society Sam 3d'
# recommendded_results = recommend(query_show_des, model)

# for index in recommendded_results:
#     print(new.iloc[index,:])

In [14]:
from sentence_transformers import models, losses, InputExample
from torch.utils.data import DataLoader
from sentence_transformers import evaluation

In [15]:
# microsoft/mpnet-base
# sentence-transformers/stsb-bert-large
word_embedding_model = models.Transformer('microsoft/mpnet-base').to(device)

Some weights of the model checkpoint at microsoft/mpnet-base were not used when initializing MPNetModel: ['lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing MPNetModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MPNetModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of MPNetModel were not initialized from the model checkpoint at microsoft/mpnet-base and are newly initialized: ['mpnet.pooler.dense.weight', 'mpnet.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predi

In [16]:
new

,movie_id,title,reviews,tags
0,19995,Avatar,Cameron’s epic can still thrill the audience w...,"In the 22nd century, a paraplegic Marine is di..."
3,49026,The Dark Knight Rises,It still might not be quite the conclusion we ...,Following the death of District Attorney Harve...
4,49529,John Carter,John Carter is a good summer movie in March bu...,"John Carter is a war-weary, former military ca..."
6,38757,Tangled,"At its finest moments, this production emanate...",When the kingdom's most wanted-and most charmi...
8,767,Harry Potter and the Half-Blood Prince,...a for fans only entry within an often distr...,"As Harry begins his sixth year at Hogwarts, he..."
...,...,...,...,...
1477,42151,Down Terrace,"A film that is compelling in its own right, wh...",After serving jail time for a mysterious crime...
1478,2292,Clerks,Clerks doesn’t feel as if it’s trying too hard...,Convenience and video store clerks Dante and R...
1480,157185,Tin Can Man,"Unique and eerie enough to be worth seeing, ev...",Recently dumped by his girlfirend for another ...
1481,14337,Primer,Time travel may provide the paradoxical mechan...,Friends/fledgling entrepreneurs invent a devic...


In [17]:
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 
                                  pooling_mode_mean_tokens=True, 
                                  pooling_mode_cls_token=False, 
                                  pooling_mode_max_tokens=False
                              )

In [18]:
# update the model
sent_model = SentenceTransformer(modules=[word_embedding_model, pooling_model]).to(device)

In [19]:
train_examples = []

for row in zip(new['tags'], new['reviews']):
    train_examples.append(InputExample(texts=[row[0], row[1]]))

In [20]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=2)
# MultipleNegativesRankingLoss not trainable because gpu limitation
# MegaBatchMarginLoss 
# MSELoss need labels
train_loss = losses.MegaBatchMarginLoss(sent_model)

In [21]:
sent_model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/474 [00:00<?, ?it/s]

In [22]:
descriptions = new['tags'].tolist()
des_embeddings = []
for i,des in enumerate(descriptions):
    des_embeddings.append(sent_model.encode(des))

In [23]:
def recommend(query, model):
    #Compute cosine-similarities with all embeddings 
    model.eval()
    query_embedd = model.encode(query)
    cosine_scores = util.pytorch_cos_sim(query_embedd, des_embeddings)
    top5_matches = torch.argsort(cosine_scores, dim=-1, descending=True).tolist()[0][0:5]
    return top5_matches

In [25]:
query_show_des = '3D Top notch education and entertainment for dinosaurs'
recommendded_results = recommend(query_show_des, sent_model)

for index in recommendded_results:
    print(new.iloc[index,:])

movie_id                                                77951
title                                  Walking With Dinosaurs
reviews     Top notch education and entertainment for dino...
tags        Walking with Dinosaurs 3D is a film depicting ...
Name: 289, dtype: object
movie_id                                               135397
title                                          Jurassic World
reviews     The visual spectacle, the dinosaur fights, the...
tags        Twenty-two years after the events of Jurassic ...
Name: 24, dtype: object
movie_id                                                  329
title                                           Jurassic Park
reviews     Jurassic Park turned out to be an incredible v...
tags        A wealthy entrepreneur secretly creates a them...
Name: 377, dtype: object
movie_id                                               105864
title                                       The Good Dinosaur
reviews     This tale of dinosaur versus Nature offers no 

In [45]:
query_show_des = 'Its Godzilla versus the paper pushers'
recommendded_results = recommend(query_show_des, sent_model)

for index in recommendded_results:
    print(new.iloc[index,:])

movie_id                                                 7278
title                                       Meet the Spartans
reviews     The worlds worst parody movie. , This is one o...
tags        From the creators of Scary Movie and Date Movi...
Name: 712, dtype: object
movie_id                                                13805
title                                          Disaster Movie
reviews     Nothing more than a 90 minute barrage of unfun...
tags        In DISASTER MOVIE, the filmmaking team behind ...
Name: 886, dtype: object
movie_id                                                82690
title                                          Wreck-It Ralph
reviews     Phil Johnston and Jennifer Lee’s script finds ...
tags        Wreck-It Ralph is the 9-foot-tall, 643-pound v...
Name: 72, dtype: object
movie_id                                                93456
title                                         Despicable Me 2
reviews     Nothing is surprising about Despicable Me 2, b

In [15]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [18]:
import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()   

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 97% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  2% |  5% |


In [38]:
device

device(type='cuda')